<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Train/Test Split and Bias and Variance

_Authors: Joseph Nelson (DC), Kevin Markham (DC)_

---

<a id="learning-objectives"></a>
<a id="learning-objectives"></a>
<a id="learning-objectives"></a>
### Learning Objectives
- Characterize the difference between error due to bias and error due to variance.
- Identify the bias-variance trade-off.
- Describe what overfitting and underfitting means in the context of model building.
- Explain problems associated with over- and underfitting.
- Grasp why train/test split is necessary.
- Explore k-folds, LOOCV, and three split methods.

# Bias

Linear regression with one input feature finds the *line* that best fits the data. This is what you want if in fact $y = \beta_0 + \beta_1 * x + \epsilon$, where $\epsilon$ is normally distributed random noise.

![](./assets/linear.png)

But if, say, $y = \beta_0 + \beta_1 x + \beta_2 x^2 +\epsilon$, then "the line that best fits the data" is *not* what you want.

![](./assets/quadratic.png)

This difference between the kind of relationship a model can capture and the kind that exists in the underlying data-generating process (the "true model") is called **bias**.

Because it is due to a fundamentally mis-specified model, *bias doesn't go away as you collect more data.*

# Variance

One way to address bias is to make your model more flexible. For instance, a model that finds the best-fitting parabola $y = \beta_0 + \beta_1 X + \beta_2 X^2$ can do everything a linear model can do (setting $\beta_2=0$) and more, so it can only be better in terms of bias.

Why stop there? Why not $y = \beta_0 + \beta_1 X + \beta_2 X^2 + \beta_3 X^3 + \beta_4 X^4 + \beta_5 X^5 + \beta_6 X^6 + \beta_7 X^7 + \beta_8 X^8$?

The problem why this approach is that more flexible models are more sensitive to that random noise $\epsilon$ in the data. They find "patterns" that aren't really there.

![](./assets/overfit.png)

Because they are sensitive to random noise in the data, overly flexible models will give very different results on datasets that are drawn from the same distribution.

![](./assets/overfit_side_by_side.png)

For this reason, we say that overly flexible models have high **variance**.

As sample size increases, even a very flexible model will learn to distinguish between signal and noise. As a result, *model variance does go away as you collect more data*.

<a id="bias-and-variance-trade-off"></a>
## Bias-Variance Tradeoff

**Exercise.** State whether each of the following changes you can make to a model would increase, decrease, or leave unchanged (1) model bias and (2) model variance, or whether its effect on those quantities cannot be determined from the information given.

- Include more of the variables that are given in your dataframe in your feature columns.

-- decrease model bias
-- variance increases

- Add the square of one of your feature columns as a new feature column.

-- decrease model bias
-- variance increases

- Replace one feature column with its logarithm.

depends which version of the variable is better fit the model

- Collect more data.

-- decreases variance
-- bias remains unchanged

- Modify the procedure you use to fit your model so that instead of minimizing the sum of squared errors, it minimizes the sum of squared errors plus a penalty term that increases with the coefficients of the model. (This approach is an example of *regularization*.)

-- u r making ur model less flexible
-- bias goes up
-- variance goes down

- Instead of linear regression, use a *generative additive model* that can fit a variety of non-linear relationships between your features and your target variable as well as linear relationships.

-- made our mode more flexible
-- mean decreasing bias
-- increasing variance

### Expressing bias and variance mathematically

Assume that there is some function $f$ such that $y=f(X)+\epsilon$ for a given input feature matrix $X$ and target variable $y$, where $\epsilon$ is normally distributed with variance $\sigma^2$. We will call this $f$ the "true model" for $y$ based on $X$. We will use $\hat{f}(X)$ ("$f$-hat of $X$") to denote a particular model's predictions for $y$ given an input feature matrix $X$.

Suppose you performed the following steps repeatedly: 

1. Draw a random sample of a given size from a particular data-generating process -- the "training set."
2. Fit a particular kind of model to that training set.
3. Draw one additional point $X^*$ from the same data-generating process and calculate the squared error of the model's prediction on that point.

The expected (long-run average) value of that squared error can be broken down into three components:

$\mbox{Expected Squared Error} = \mbox{Bias}^2 + \mbox{Variance} + \mbox{Irreducible Error}$

where
- $\mbox{Expected Squared Error} = E[(y-\hat{f}(X^*))^2]$
    - (expected squared difference between the target value and the model's prediction)
- $\mbox{Bias} = E[\hat{f}(X^*) - f(X^*)]$
    - (expected difference between a prediction from the model and a prediction from the true model)
- $\mbox{Variance} = E[(\hat{f}(X^*) - E[\hat{f}(X^*)])^2]$
    - (expected squared difference between the model's prediction and its expected prediction)
- $\mbox{Irreducible Error} = \sigma^2$
    - (noise in the data-generating mechanism)

### Example

Suppose you build a linear model to predict the weights of golden retrievers when they are one year old based on their weights when they are seven weeks old.

**Bias:** Suppose that the relationship between seven-week weight and one-year weight has a bit of an S-shape, because very large and very small puppies tend to regress toward the mean. *The mismatch between the true s-shaped model and the assumed linear model constitutes bias.*

**Variance:** Because your sample size is finite, each time you train the model on new data you don't quite nail even the linear relationship that's closest to the true model. *The random departures of your model from this best possibility given your bias constitute variance.*

**Irreducible error:** Some 10-pound puppies will grow up to be 70 pounds, while others will grow up to be 90 pounds. In general, one-year weight is not perfectly predictable from seven-week weight. As a  result, *even the true model will have some irreducible error.*

**Exercise.**

- Would a "true model" $f(X)$ make perfect predictions? Why or why not?

- **Challenge:** How could you create a model for predicting some $y$ that does even better than the true model for $y$ based on your input features $X$?

<a id="exploring-the-bias-variance-tradeoff"></a>
### Exploring the Bias-Variance Trade-Off

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

# Allow plots to appear in the notebook.
%matplotlib inline

<a id="brain-and-body-weight-mammal-dataset"></a>
### Brain and Body Weight Mammal Data Set

This is a [data set](http://people.sc.fsu.edu/~jburkardt/datasets/regression/x01.txt) of the average weight of the body (in kg) and the brain (in g) for 62 mammal species. We'll use this dataset to investigate bias vs. variance. Let's read it into Pandas and take a quick look:

In [ ]:
import os

path = os.path.join('..', 'data', 'mammals.txt')
cols = ['brain','body']
mammals = pd.read_csv(path, sep='\t', names=cols, header=0)
mammals.head()

We're going to focus on a smaller subset in which the body weight is less than 200 kg.

In [ ]:
# Only keep rows in which the body weight is less than 200 kg.


Let's create a scatterplot (using [Seaborn](http://stanford.edu/~mwaskom/software/seaborn/)) to visualize the relationship between brain and body weight:

There appears to be a relationship between brain and body weight for mammals.

<a id="making-a-prediction"></a>
### Making a Prediction

<a id="linear-regression-quick-review"></a>
#### Linear Regression: A Quick Review

<img src="./assets/linear-residuals.png" width=800 /img>

Now let's pretend that a **new mammal species** is discovered. We measure the body weight of every member of this species we can find and calculate an **average body weight of 100 kgs**. We want to **predict the average brain weight** of this species (rather than measuring it directly). How might we do this?

We drew a straight line that appears to best capture the relationship between brain and body weight. So, we might predict that our new species has a brain weight of about 45 g, as that's the approximate y value when x=100.

This is known as a "linear model" or a "linear regression model."

<a id="making-a-prediction-from-a-sample"></a>
## Making a Prediction From a Sample

The point of supervised learning is to generalize from the samples we have to samples we don't have. Let's simulate this situation by assigning each of the 51 observations to **either universe 1 or universe 2**:

In [ ]:
# Set a random seed for reproducibility.
np.random.seed(12345)

# Randomly assign every observation to either universe 1 or universe 2.


We can now tell Seaborn to create two plots in which the left plot only uses the data from **universe 1** and the right plot only uses the data from **universe 2**:

In [ ]:
# col='universe' subsets the data by universe and creates two separate plots.


The line looks pretty similar between the two plots, despite the fact that they used separate samples of data. In both cases, we would predict a brain weight of about 45 g.

It's easier to see the degree of similarity by placing them on the same plot:

In [ ]:
# hue='universe' subsets the data by universe and creates a single plot.


**Exercise.** How would you characterize the bias and variance of a linear model for brain size against body size? Justify your answer.

<a id="lets-try-something-completely-different"></a>
### Let's Try Something Completely Different

What would a **low-bias, high-variance** model look like? Let's try polynomial regression with an eighth-order polynomial.

- It's **low bias** because the models match the data effectively.
- It's **high variance** because the models are widely different, depending on which observations happen to be available in that universe. (For a body weight of 100 kg, the brain weight prediction would be 40 kg in one universe and 0 kg in the other!)

<a id="balancing-bias-and-variance"></a>
## Balancing Bias and Variance
Can we find a middle ground?

Perhaps we can create a model that has **less bias than the linear model** and **less variance than the eighth order polynomial**?

Let's try a second order polynomial instead:

This seems better. In both the left and right plots, **it fits the data well, but not too well**.

This is the essence of the **bias-variance trade-off**: You are seeking a model that appropriately balances bias and variance and thus will generalize to new data (known as "out-of-sample" data).

**Exercise.**

- Why do we care about variance if we know we can generate a more accurate model with higher complexity?

- **Challenge:** Is it possible for a model that learns from the data to have zero bias and zero variance on a data set that has irreducible error?

<a id="train-test-split"></a>
## Model Evaluation

<a id="evaluation-procedure--train-and-test-on-the-entire-dataset-do-not-do-this"></a>
### Evaluation Procedure #1: Train and Test on the Entire Data Set (Do Not Do This)

1. Train the model on the **entire data set**.
2. Test the model on the **same data set** and evaluate how well we did by comparing the **predicted** response values with the **true** response values.

#### Load in the Boston data.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston

boston = load_boston()

In [ ]:
print(boston.DESCR)

#### Create X and y variables to store the feature matrix and response from the Boston data.

In [ ]:
# Create a DataFrame for both parts of data; don't forget to assign column names.
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = pd.DataFrame(boston.target, columns=['MEDV'])

#### Concatenate y and X, then overwrite the Boston variable.

#### Perform basic EDA to make sure the data are in order.

In [ ]:
# Count null values of each variable


In [ ]:
# Make sure data is numeric


In [ ]:
# Look at basic descriptive statistics


#### Prepare a feature matrix (X) and response (y)  for scikit-learn.

#### Import linear regression, instantiate, fit, and preview predictions.

#### Store the predicted response values.

We need a measure of model performance. The most common choices for regression problems are:

- **R-squared**: The percentage of variance captured by the model.
- **Mean squared error**: The average squared distance between the prediction and the correct answer.

We'll use mean squared error.

#### Compute mean squared error using a function from `metrics`.

This is known as the **training mean squared error** because we are evaluating the model based on the same data we used to train the model.

<a id="problems-with-training-and-testing-on-the-same-data"></a>
### Problem With Training and Testing on the Same Data

Maximizing the MSE on **in-sample data** rewards high-variance/low-bias models that fit the noise in the training data as well as the signal.

Our goal is to estimate likely performance of a model on **out-of-sample data**, so a better approach is to put some data aside as a test set.

<a id="evaluation-procedure--traintest-split"></a>
### Evaluation procedure #2: Train/Test Split

1. Split the data set into two pieces: a **training set** and a **testing set**.
2. Train the model on the **training set**.
3. Test the model on the **testing set** and evaluate how well we did.

A common rule of thumb is to set aside 30% of your data set for testing.

**Testing MSE is a better estimate of out-of-sample performance than training MSE.**

#### Understanding the `train_test_split` Function

![train_test_split](./assets/train_test_split.png)

#### Understanding the `random_state` Parameter

The `random_state` is a pseudo-random number that allows us to reproduce our results every time we run them. However, it makes it impossible to predict what are exact results will be if we chose a new `random_state`.

`random_state` is very useful for testing that your model was made correctly since it provides you with the same split each time. However, make sure you remove it if you are testing for model variability!

In [ ]:
# WITHOUT a random_state parameter:
#  (If you run this code several times, you get different results!)
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Print the first element of each object.
print(X_train.head(1))

In [ ]:
# WITH a random_state parameter:
#  (Same split every time! Note you can change the random state to any integer.)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Print the first element of each object.
print(X_train.head(1))
print(X_test.head(1))
print(y_train.head(1))
print(y_test.head(1))

#### Step 1: Split X and y into training and testing sets (using `random_state` for reproducibility).

#### Step 2: Train the model on the training set.

#### Step 3: Test the model on the testing set and check the accuracy.

![Bias-variance tradeoff](./assets/bias_variance.png)

#### Go back to Step 1 and try adding new variables and transformations.

- **Training error**: Decreases as model complexity increases (lower value of k).
- **Testing error**: Is minimized at the optimum model complexity.

<a id="comparing-test-performance-with-a-null-baseline"></a>
### Comparing Test Performance With a Null Baseline

How do you know if your model is any good?

It helps to have benchmarks to compare against. A very simple model can provide a *baseline* that you can use to check whether your model is complete garbage or not.

A good baseline for linear regression is a model that just predicts the mean of the target variable every time.

In [ ]:
# Use .apply() to broadcast a mean for every prediction.


**Exercise.** What does this result tell us about our linear regression model?

<a id="k-folds-cross-validation"></a>
## K-Folds Cross-Validation
---

A simple train/test split has at least two drawbacks:

- Setting aside a test set that is large enough to provide a good estimate of performance can get in the way of training and lead you to choose a simpler model than you would need to do well when training on the full data set.
- Evaluating many models on the same test set can lead to overfitting on the test set.

**Solution:** Train the model on k separate train-test-splits, then average the resulting evaluation metric values.

![](./assets/cross_validation_diagram.png)

This approach allows you to make each test set smaller while still getting a good estimate of accuracy and makes it harder to overfit to the test set by using all of the test set for evaluation.

<a id="leave-one-out-cross-validation"></a>
### Leave-One-Out Cross-Validation

An extreme case of k-fold cross-validation is leave-one-out cross-validation, where you hold out just one data point at a time.

This approach is great for making the best use of the training data while getting a good estimate of performance. However, it can be *very slow* because it requires fitting your model $n$ times.

A more common approach is to set $k$ somewhere between 5 and 10.

<a id="intro-to-cross-validation-with-the-boston-data"></a>
### Intro to Cross-Validation With the Boston Data

#### Create a cross-validation with five folds.

<a id="three-way-data-split"></a>
## Three-Way Data Split
---
Goodhart's law: "When a measure becomes a target, it ceases to be a good measure."

Cross-validation performance provides an upwardly biased estimate of out-of-sample performance when you use it to select a model, even with $k$-fold cross validation.

If you care about being able to estimate model performance after tuning by cross-validation, then you should set aside a true test set from the beginning that you don't use even in cross-validation.

![](./assets/Train-Test-Split-CV.png)

- If you care about having an unbiased estimate of model performance when you are done, then you should split your data three ways:
    - **Training set**: A set of examples used for learning – what parameters of the classifier?
    - **Validation set**: A set of examples used to tune the parameters of the classifier.
    - **Testing set**: A set of examples used ONLY to assess the performance of the fully trained classifier.
- The training-validation split can be k-fold.
- Validation and testing must be separate data sets. Once you have the final model set, you cannot do any additional tuning after testing.

**Steps for a three-way split**

1. Set aside a testing set.
2. Select architecture (model type) and training parameters ($k$).
3. Train the model using the training set and evaluate it using the validation set (using either simple or k-fold train/test split).
4. Repeat, selecting different architectures (models) and tuning parameters.
5. Select the best model.
6. Assess the model with the final testing set.

<a id="additional-resources"></a>
<a id="additional-resources"></a>
### Additional Resources
- [Bias Variance](http://scott.fortmann-roe.com/docs/BiasVariance.html)
- University of Washington [slides](https://courses.cs.washington.edu/courses/cse546/12wi/slides/)

<a id="summary"></a>
### Summary

In this lab, we compared four methods of estimating model accuracy on out-of-sample data:

1. **Train on the entire dataset**
2. **Simple train/test split**
3. **K-folds cross validation**
4. **Three-way split**

# Questions?

# [Exit Tickets](https://docs.google.com/forms/d/1BW4rVsCx8Nzp3q2B7SQ_tL1xqKZr4GGoQ5qeZfayxh4/viewform?ts=5ad40144&edit_requested=true)